In [1]:
 # Google Colab doesn't come with this package installed, so we'll install it.
!pip install keras-tuner

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 135 kB 24.5 MB/s 
     |████████████████████████████████| 1.6 MB 10.2 MB/s 


In [2]:
# Import our dependencies

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

In [4]:
#  Import and read the charity_data.csv

df = pd.read_csv("charity_data.csv")
print(df.info())
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34299 entries, 0 to 34298
Data columns (total 12 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   EIN                     34299 non-null  int64 
 1   NAME                    34299 non-null  object
 2   APPLICATION_TYPE        34299 non-null  object
 3   AFFILIATION             34299 non-null  object
 4   CLASSIFICATION          34299 non-null  object
 5   USE_CASE                34299 non-null  object
 6   ORGANIZATION            34299 non-null  object
 7   STATUS                  34299 non-null  int64 
 8   INCOME_AMT              34299 non-null  object
 9   SPECIAL_CONSIDERATIONS  34299 non-null  object
 10  ASK_AMT                 34299 non-null  int64 
 11  IS_SUCCESSFUL           34299 non-null  int64 
dtypes: int64(4), object(8)
memory usage: 3.1+ MB
None


,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [5]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.

df = df.drop(columns=["EIN", "NAME"])

In [6]:
# Determine the number of unique values in each column.

for col in df.columns:
    print(f"{col} \n{df[col].value_counts()}\n\n")

APPLICATION_TYPE 
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64


AFFILIATION 
Independent         18480
CompanySponsored    15705
Family/Parent          64
National               33
Regional               13
Other                   4
Name: AFFILIATION, dtype: int64


CLASSIFICATION 
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64


USE_CASE 
Preservation     28095
ProductDev        5671
CommunityServ      384
Heathcare          146
Other                3
Name: USE_CASE, dtype: int64


ORGANIZATION 
Trust           23515
Association     10255
Co-operative      486
Corporation        43
Name: ORGANIZATION, dtype: int6

In [7]:
# Look at APPLICATION_TYPE value counts for binning

df["APPLICATION_TYPE"].value_counts()

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [8]:
application_types_to_replace = (df["APPLICATION_TYPE"].value_counts().loc[lambda x: x < 500]).keys().tolist()

In [9]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`

application_types_to_replace

['T9', 'T13', 'T12', 'T2', 'T25', 'T14', 'T29', 'T15', 'T17']

In [10]:
# Replace in dataframe

for code in application_types_to_replace:
    print(code)
    df["APPLICATION_TYPE"] = df["APPLICATION_TYPE"].replace(code, "Other")
        
# Check to make sure binning was successful

df["APPLICATION_TYPE"].value_counts()

T9
T13
T12
T2
T25
T14
T29
T15
T17


T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [11]:
# Look at CLASSIFICATION value counts for binning

df["CLASSIFICATION"].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [12]:
# You may find it helpful to look at CLASSIFICATION value counts >1

classifications_to_replace = (df["CLASSIFICATION"].value_counts().loc[lambda x: x < 1000]).keys().tolist()

In [13]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`

# Replace in dataframe

for code in classifications_to_replace:
    df["CLASSIFICATION"] = df["CLASSIFICATION"].replace(code, "Other")
    
# Check to make sure binning was successful

df["CLASSIFICATION"].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [14]:
# Convert categorical data to numeric with `pd.get_dummies`

alphasoup_cat = df.dtypes[df.dtypes == "object"].index.tolist() 
alphasoup_cat

['APPLICATION_TYPE',
 'AFFILIATION',
 'CLASSIFICATION',
 'USE_CASE',
 'ORGANIZATION',
 'INCOME_AMT',
 'SPECIAL_CONSIDERATIONS']

In [15]:
df[alphasoup_cat].nunique() 

APPLICATION_TYPE          9
AFFILIATION               6
CLASSIFICATION            6
USE_CASE                  5
ORGANIZATION              4
INCOME_AMT                9
SPECIAL_CONSIDERATIONS    2
dtype: int64

In [16]:
df1 = pd.get_dummies(df["APPLICATION_TYPE"], prefix="APP_TYPE", prefix_sep='_')
df2 = pd.get_dummies(df["AFFILIATION"], prefix="AFFIL", prefix_sep='_')
df3 = pd.get_dummies(df["CLASSIFICATION"], prefix="CLASS", prefix_sep='_')
df4 = pd.get_dummies(df["USE_CASE"], prefix="USE_CASE", prefix_sep='_')
df5 = pd.get_dummies(df["ORGANIZATION"], prefix="ORG", prefix_sep='_')
df6 = pd.get_dummies(df["INCOME_AMT"], prefix="INCOME", prefix_sep='_')
df7 = pd.get_dummies(df["SPECIAL_CONSIDERATIONS"], prefix="SPECIAL_CONSID", prefix_sep='_')
df8 = pd.concat([df[["ASK_AMT", "IS_SUCCESSFUL", "STATUS"]]], axis=1)

In [17]:
enc_df = pd.concat([df1, df2, df3, df4, df5, df6, df7, df8], axis=1)
enc_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34299 entries, 0 to 34298
Data columns (total 44 columns):
 #   Column                  Non-Null Count  Dtype
---  ------                  --------------  -----
 0   APP_TYPE_Other          34299 non-null  uint8
 1   APP_TYPE_T10            34299 non-null  uint8
 2   APP_TYPE_T19            34299 non-null  uint8
 3   APP_TYPE_T3             34299 non-null  uint8
 4   APP_TYPE_T4             34299 non-null  uint8
 5   APP_TYPE_T5             34299 non-null  uint8
 6   APP_TYPE_T6             34299 non-null  uint8
 7   APP_TYPE_T7             34299 non-null  uint8
 8   APP_TYPE_T8             34299 non-null  uint8
 9   AFFIL_CompanySponsored  34299 non-null  uint8
 10  AFFIL_Family/Parent     34299 non-null  uint8
 11  AFFIL_Independent       34299 non-null  uint8
 12  AFFIL_National          34299 non-null  uint8
 13  AFFIL_Other             34299 non-null  uint8
 14  AFFIL_Regional          34299 non-null  uint8
 15  CLASS_C1000        

In [18]:
# Split our preprocessed data into our features and target arrays

y = enc_df["IS_SUCCESSFUL"].values
X = enc_df.drop(["IS_SUCCESSFUL"], axis=1).values

# Split the preprocessed data into a training and testing dataset

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0) 

In [19]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

print(X_train_scaled.shape)
print(X_test_scaled.shape)

(25724, 43)
(8575, 43)


In [23]:
# Create a method that creates a new Sequential model with hyperparameter options
def create_model(hp):
    
    # Instantiate a Sequential model
    nn_model = tf.keras.models.Sequential()

    # Allow kerastuner to decide which activation function to use in hidden layers.
    activation = hp.Choice('activation', ['relu','tanh','sigmoid'])
    
    # Allow kerastuner to decide the number of neurons in first layer and also
    # the activation function. 
    nn_model.add(tf.keras.layers.Dense(units=hp.Int('first_units',
        min_value=1,
        max_value=10,
        step=2), activation=activation, input_dim=43))

    # Allow kerastuner to decide the number of hidden layers and number of 
    # neurons in each one
    for i in range(hp.Int('num_layers', 1, 6)):
        nn_model.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
            min_value=1,
            max_value=10,
            step=2),
            activation=activation))
    
    # Define the output layer
    nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

    # Compile the model
    nn_model.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])
    
    return nn_model

In [24]:
# Import the kerastuner library

import keras_tuner as kt

tuner = kt.Hyperband(
    create_model,
    objective="val_accuracy",
    max_epochs=20,
    hyperband_iterations=2)

In [25]:
# Run the kerastuner search for best hyperparameters

tuner.search(X_train_scaled, y_train,epochs=20, validation_data=(X_test_scaled, y_test))

Trial 60 Complete [00h 00m 32s]
val_accuracy: 0.6468804478645325

Best val_accuracy So Far: 0.7272303104400635
Total elapsed time: 00h 14m 37s


In [26]:
 # Get best model hyperparameters
 
best_hyper = tuner.get_best_hyperparameters(1)[0]
best_hyper.values

{'activation': 'tanh',
 'first_units': 3,
 'num_layers': 3,
 'units_0': 5,
 'units_1': 9,
 'units_2': 1,
 'units_3': 7,
 'units_4': 3,
 'units_5': 9,
 'tuner/epochs': 20,
 'tuner/initial_epoch': 7,
 'tuner/bracket': 1,
 'tuner/round': 1,
 'tuner/trial_id': '0021'}

In [27]:
 # Evaluate best model against full test data
 
best_model = tuner.get_best_models(1)[0]
model_loss, model_accuracy = best_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5650 - accuracy: 0.7272 - 411ms/epoch - 2ms/step
Loss: 0.5649535059928894, Accuracy: 0.7272303104400635


In [28]:
 # Summarize the best model
 
best_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 3)                 132       
                                                                 
 dense_1 (Dense)             (None, 5)                 20        
                                                                 
 dense_2 (Dense)             (None, 9)                 54        
                                                                 
 dense_3 (Dense)             (None, 1)                 10        
                                                                 
 dense_4 (Dense)             (None, 1)                 2         
                                                                 
Total params: 218
Trainable params: 218
Non-trainable params: 0
_________________________________________________________________


In [29]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

nn = tf.keras.models.Sequential()

# First hidden layer

nn.add(tf.keras.layers.Dense(units=3, input_dim=43, activation="tanh"))

# Second hidden layer

nn.add(tf.keras.layers.Dense(units=5, activation="relu"))

# Third hidden layer

nn.add(tf.keras.layers.Dense(units=9, activation="relu"))

# Fourth hidden layer

nn.add(tf.keras.layers.Dense(units=1, activation="relu"))

# Output layer

nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model

nn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_5 (Dense)             (None, 3)                 132       
                                                                 
 dense_6 (Dense)             (None, 5)                 20        
                                                                 
 dense_7 (Dense)             (None, 9)                 54        
                                                                 
 dense_8 (Dense)             (None, 1)                 10        
                                                                 
 dense_9 (Dense)             (None, 1)                 2         
                                                                 
Total params: 218
Trainable params: 218
Non-trainable params: 0
_________________________________________________________________


In [30]:
# Compile the model

nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"]) 

In [31]:
# Train the model

fit_model = nn.fit(X_train_scaled, y_train, epochs=100) 

Epoch 1/100
804/804 [==============================] - 2s 2ms/step - loss: 0.6242 - accuracy: 0.6904
Epoch 2/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5864 - accuracy: 0.7238
Epoch 3/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5779 - accuracy: 0.7264
Epoch 4/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5739 - accuracy: 0.7281
Epoch 5/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5709 - accuracy: 0.7300
Epoch 6/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5698 - accuracy: 0.7303
Epoch 7/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5689 - accuracy: 0.7315
Epoch 8/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5687 - accuracy: 0.7310
Epoch 9/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5681 - accuracy: 0.7303
Epoch 10/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5679 - accura

In [32]:
# Evaluate the model using the test data

model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5808 - accuracy: 0.7215 - 372ms/epoch - 1ms/step
Loss: 0.5808143615722656, Accuracy: 0.7215160131454468


In [33]:
# Export our model to HDF5 file

nn.save("nn_optimized.h5")